In [ ]:
## (제스처마다) 데이터프레임 읽기 - X,y 분리 - 시퀀스데이터로 - 훈련데이터 시험데이터 분리 - 
## (for 나와서) X,y concat - 모델 작성 - 학습

In [10]:
import os
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

path = "./Dataset"
file_list = os.listdir(path)

actions =[]
for i in range(len(file_list)) :
    actions.append(file_list[i][:-4])

seq_length = 10

In [12]:
# 제스처별로 전처리하고 훈련데이터 시험데이터 생성하기
datax, datay = [], []

# for i in range(1):
for i in range(len(file_list)):

    # 데이터프레임 읽고 변수, 라벨 분리
    df = pd.read_csv(f'{path}/{file_list[i]}', header=None)
    dfx = df.iloc[:, :-1].to_numpy()
    dfy = df.iloc[:, -1].to_numpy()

    # 시퀀스길이에 따라 시퀀스데이터로 변환
    for i in range(len(df) - seq_length):
        datax.append(dfx[i: i + seq_length])
        datay.append(dfy[i + seq_length])



In [13]:
# 정답지 원-핫 인코딩
datay = to_categorical(datay, num_classes=len(actions))

In [15]:
xtrain, xtest, ytrain, ytest = train_test_split(datax, datay, test_size=0.1)

In [18]:
from tensorflow import keras

model = keras.Sequential()
model.add( keras.layers.SimpleRNN(10, activation = 'relu', input_shape = xtrain.shape[1:3]) )    # 10행 5열
model.add( keras.layers.Dropout(0.1) )
model.add( keras.layers.Dense(1) )    # 그냥 예측이므로 시그모이드, 소프트맥스 아님

model.compile(optimizer='adam', loss='categorical_crossentropy')    # 최적화함수, 오차계산방법

checkpoint = keras.callbacks.ModelCheckpoint('Models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, verbose=1, mode='auto')
early = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.fit(xtrain, ytrain,
            validation_data = (xtest, ytest),
            epochs = 100,
            callbacks = [checkpoint, reduceLR, early])

AttributeError: 'list' object has no attribute 'shape'